In [1]:
print("hi")

hi


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# 📁 Paths
detection_cfg = "weights/detection/yolov3-detection.cfg"
detection_weights = "weights/detection/yolov3-detection_final.weights"
ocr_cfg = "weights/ocr/yolov3-ocr.cfg"
ocr_weights = "weights/ocr/yolov3-ocr_final.weights"

# 📦 Load Detection Model
detection_net = cv2.dnn.readNetFromDarknet(detection_cfg, detection_weights)
detection_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
detection_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# 📦 Load OCR Model
ocr_net = cv2.dnn.readNetFromDarknet(ocr_cfg, ocr_weights)
ocr_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
ocr_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# 📂 Dataset folder (images and labels)
test_images_dir = "data/test/images"
test_labels_dir = "data/test/labels"  # Assuming YOLO txt format

# 📌 Load class names for OCR (digits + Arabic letter)
ocr_classes = [str(i) for i in range(10)] + ['|', 'أ']  # Adapt as needed

def get_output_layers(net):
    layer_names = net.getLayerNames()
    return [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# 🔍 Inference + Evaluation
y_true = []
y_pred = []

for filename in os.listdir(test_images_dir):
    if not filename.endswith('.jpg'):
        continue
    img_path = os.path.join(test_images_dir, filename)
    image = cv2.imread(img_path)
    height, width = image.shape[:2]

    # ➤ Run OCR model
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    ocr_net.setInput(blob)
    layer_outputs = ocr_net.forward(get_output_layers(ocr_net))

    boxes, confidences, class_ids = [], [], []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.5:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(conf))
                class_ids.append(class_id)

    # Apply NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    pred = [ocr_classes[class_ids[i]] for i in indices.flatten()]
    
    # Get ground truth
    label_path = os.path.join(test_labels_dir, filename.replace(".jpg", ".txt"))
    gt = []
    with open(label_path, 'r') as f:
        for line in f:
            gt.append(ocr_classes[int(line.split()[0])])

    y_true.extend(gt)
    y_pred.extend(pred)

# 📈 Metrics
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

# 🔲 Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=ocr_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ocr_classes)
disp.plot(xticks_rotation='vertical')
plt.title("Confusion Matrix (OCR Module)")
plt.show()
